# Univariate and Bivariate Von Mises distributions implemented in Numpyro

In [1]:
import jax
import jax.numpy as np
import jax.random as random
from jax.scipy.special import gammaln, logsumexp
from jax import jit, lax
from numpyro.distributions import constraints 
from numpyro.distributions.distribution import Distribution 
from numpyro.util import copy_docs_from
from functools import partial
from numpyro.distributions.util import lazy_property, promote_shapes, validate_sample
from functools import reduce

"""The implementation of the univariate Von Mises distribution follows the implementation of the same
distribution in Pytorch torch.distributions package. The original code for the Pytorch implementation can be 
found athttps://pytorch.org/docs/stable/_modules/torch/distributions/von_mises.html#VonMises"""

_I0_COEF_SMALL = np.array([1.0, 3.5156229, 3.0899424, 1.2067492, 0.2659732, 0.360768e-1, 0.45813e-2])
_I0_COEF_LARGE = np.array([0.39894228, 0.1328592e-1, 0.225319e-2, -0.157565e-2, 0.916281e-2,
                  -0.2057706e-1, 0.2635537e-1, -0.1647633e-1, 0.392377e-2])
_I1_COEF_SMALL = np.array([0.5, 0.87890594, 0.51498869, 0.15084934, 0.2658733e-1, 0.301532e-2, 0.32411e-3])
_I1_COEF_LARGE = np.array([0.39894228, -0.3988024e-1, -0.362018e-2, 0.163801e-2, -0.1031555e-1,
                  0.2282967e-1, -0.2895312e-1, 0.1787654e-1, -0.420059e-2])

@jit
def _log_modified_bessel_fn(x, order):
    """
    Returns ``log(I_order(x))`` for ``x > 0``,
    where `order` is either 0 or 1.
    
    Based on https://pytorch.org/docs/stable/_modules/torch/distributions/von_mises.html#VonMises
    """
    # compute small solution
    y = (x / 3.75)
    y = y * y
    
    COEF_SMALL = np.where(np.ones((7,))*order, _I1_COEF_SMALL, _I0_COEF_SMALL)
    COEF_LARGE = np.where(np.ones((9,))*order, _I1_COEF_LARGE, _I0_COEF_LARGE)
    
    small = _eval_poly_small(y, COEF_SMALL)
    small = np.where(np.ones(x.shape)*order, abs(x) * small, small)
    small = np.log(small)

    # compute large solution
    y = 3.75 / x
    large = x - 0.5 * np.log(x) + np.log(_eval_poly_large(y, COEF_LARGE))

    result = np.where(x < 3.75, small, large)
    return result

@jit
def _eval_poly_small(y, coef):
    return coef[-7] + y*(coef[-6] + y*(coef[-5] + y*(coef[-4] + y*(coef[-3] + y*(coef[-2] + y*coef[-1])))))

@jit
def _eval_poly_large(y, coef):
    return coef[-9] + y*(coef[-8] + y*(coef[-7] + y*( coef[-6] + 
                        y*(coef[-5] + y*(coef[-4] + y*(coef[-3] + y*(coef[-2] + y*coef[-1])))))))

@jit
def condition(args):
    return ~np.all(args[2])

@jit
def loop(args):
    x, proposal_r, done, key, concentration = args
    key, subkey = jax.random.split(key)
    u = jax.random.uniform(subkey, shape = (3,) + x.shape)
    u1, u2, u3 = u.squeeze()
    z = np.cos(np.pi * u1)
    f = (1 + proposal_r * z) / (proposal_r + z)
    c = concentration * (proposal_r - f)
    accept = ((c * (2 - c) - u2) > 0) | (np.log(c / u2) + 1 - c >= 0)
    x = np.where(accept, np.sign(u3 - 0.5) * np.arccos(f),  x)          
    done = done | accept
    return x, proposal_r, done, key, concentration

@jit
def _rejection_sample(loc, concentration, proposal_r, key, x):
    """
    Acceptance-rejection sampling method – translated from the Pytorch Von Mises implementation. 
    
    The sampling algorithm for the von Mises distribution is based on the following paper:
    Best, D. J., and Nicholas I. Fisher.
    "Efficient simulation of the von Mises distribution." Applied Statistics (1979): 152-157.
    """
    done = np.zeros(x.shape, dtype=bool)
    x = lax.while_loop(condition, loop, (x, proposal_r, done, key, concentration))[0]
    return (x + np.pi + loc) % (2 * np.pi) - np.pi
    
@copy_docs_from(Distribution)
class VonMises(Distribution):
    """
    A circular von Mises distribution.

    This implementation uses polar coordinates. The ``loc`` and ``value`` args
    can be any real number (to facilitate unconstrained optimization), but are
    interpreted as angles modulo 2 pi.

    :param int or ndarray: an angle in radians.
    :param int ord ndarray: concentration parameter
    
    Based on https://pytorch.org/docs/stable/_modules/torch/distributions/von_mises.html#VonMises
    """
    arg_constraints = {'loc': constraints.real, 'concentration': constraints.positive}
    support = constraints.real
    has_rsample = False
    
    def __repr__(self):
        return str(type(self).__name__) + "(loc: " + str(self.loc) + ", concentration: " + str(self.concentration) + ")"

    def __init__(self, loc, concentration, validate_args=None):
        self.loc, self.concentration = promote_shapes(loc, concentration)
        batch_shape = lax.broadcast_shapes(np.shape(loc), np.shape(concentration))
        event_shape = ()
        tau = 1 + np.sqrt(1 + 4 * self.concentration ** 2)
        rho = (tau - np.sqrt(2 * tau)) / (2 * self.concentration)
        self._proposal_r = (1 + rho ** 2) / (2 * rho)
        super(VonMises, self).__init__(batch_shape, event_shape, validate_args)

    def log_prob(self, value):
        log_prob = self.concentration * np.cos(value - self.loc)
        log_prob = log_prob - np.log(2 * np.pi) - _log_modified_bessel_fn(self.concentration, 0)
        return log_prob.T
    
    def sample(self, key, sample_shape = ()):
        """
        The sampling algorithm for the von Mises distribution is based on the following paper:
        Best, D. J., and Nicholas I. Fisher.
        "Efficient simulation of the von Mises distribution." Applied Statistics (1979): 152-157.
        
        Based on https://pytorch.org/docs/stable/_modules/torch/distributions/von_mises.html#VonMises
        """
        if isinstance(sample_shape, int):
            shape = tuple([sample_shape]) + self.batch_shape + self.event_shape
        else:
            shape = sample_shape + self.batch_shape + self.event_shape
        x = np.empty(shape)            
        return _rejection_sample(self.loc, self.concentration, self._proposal_r, key, x)

    def expand(self, batch_shape):
        """
        Function to initialize batch_shape number of parallel distributions
        """
        validate_args = self.__dict__.get('_validate_args')
        loc = np.ones(batch_shape)*self.loc
        concentration = np.ones(batch_shape)*self.concentration
        return VonMises(loc, concentration, validate_args=validate_args)

    @property
    def mean(self):
        """
        The provided mean is the circular one.
        """
        return self.loc

    @lazy_property
    def variance(self):
        """
        The provided variance is the circular one.
        """
        return 1 - np.exp(_log_modified_bessel_fn(self.concentration, 1) -
                    _log_modified_bessel_fn(self.concentration, 0))

## Bivariate Von Mises – sampling & logCinv 

In [2]:
@partial(jit, static_argnums = (0,1,))
def _acg_bound(sample, k1, k2, alpha, key):
    lam = np.concatenate((np.zeros([len(k1), 1]), 0.5*(k1 - alpha**2/k2).reshape(len(k1), 1)), axis = 1)
    lambda_min = np.min(lam, axis = 1).reshape([len(k1), 1])
    lam = lam - lambda_min
    b_values = np.concatenate((np.array(np.sqrt(lam[:,1]**2 + 1) - lam[:,1]
                                        + 1).reshape([len(k1), 1]), np.ones([len(k1),1])*2), axis = 1)
    b0 = np.min(b_values, axis = 1).reshape([len(k1), 1])

    phi = 1 + 2*lam/b0
    den = _log_modified_bessel_fn(k2, 0)
    accept = np.zeros((sample.shape[0], k1.shape[0], 1), dtype = "bool_")
    args = (key, phi, k1, k2, lam, lambda_min, sample, accept, den, b0, alpha)
    res = lax.while_loop(loop_condition, loop_acg, args)[6]
    
    return np.arctan2(res[...,1], res[...,0])

@jit
def loop_acg(args):
    key, phi, k1, k2, lam, lambda_min, sample, accept, den, b0, alpha = args
    key, subkey = jax.random.split(key)
    x = np.where(accept, 0,  jax.random.normal(subkey, sample.shape) * np.sqrt(1/phi))    
    r = np.sqrt(np.sum(x**2, axis = -1))        
    r = np.expand_dims(r, axis=-1)
    x = x/r
    u = (x**2 * lam).sum(-1)
    v = jax.random.uniform(subkey, (sample.shape[0], k1.shape[0]))

    logf = k1*(x[...,0] - 1) + lambda_min.T + _log_modified_bessel_fn(np.sqrt(k2**2 + alpha**2 * x[...,1]**2), 0) - den           
    loggi = 0.5 * (2 - b0.T) + np.log(1 + 2*u/b0.T) + np.log(b0.T/2)
    logfg = np.add(logf, loggi)
    logfg = logfg.reshape([sample.shape[0], k1.shape[0]])
    
    accept = v < np.exp(logfg)
    accept = accept[..., None]
    sample = np.where(accept, x,  sample)

    return (key, phi, k1, k2, lam, lambda_min, sample, accept, den, b0, alpha)

@jit
def loop_condition(args):
    return np.count_nonzero(np.isnan(args[6])) > 0

In [3]:
@partial(jit, static_argnums = (1,))
def log_im(order, x): 
    """ x is a parameter, like k1 or k2
        Based on '_log_modified_bessel_fn'
        Tanabe, A., Fukumizu, K., Oba, S., Takenouchi, T., & Ishii, S. (2007). 
        Parameter estimation for von Mises–Fisher distributions. Computational Statistics, 22(1), 145-157. 
    """
    if not isinstance(x, int):
        x = x.reshape([len(x), 1, 1])   
    s = np.arange(0 , 251).reshape(251, 1)  
    fs = 2 * s * np.log(x/2) - gammaln(s + 1) - gammaln(order + s + 1)
    f_max = np.max(fs, axis = -2)
    if hasattr(x, '__len__'):
        x = x.reshape([len(x), 1])   
   
    return (order * np.log(x/2) + f_max + logsumexp(fs - f_max[:,None], -2)).squeeze()

In [4]:
@partial(jit, static_argnums = (0,))
def log_C(k1, k2, lam):
    
    """Harshinder Singh, Vladimir Hnizdo, and Eugene Demchuk
    Probabilistic model for twodependent circular variables.
    Biometrika, 89(3):719–723, 2002.

    Closed form expression of the normalizing constant
    Vectorized and in log-space
    
    k1, k2 & lam are the parameters from the bivariate von Mises
    
    Since the closed expression is an infinite sum, 'terms' is the number
    of terms, over which the expression is summed over. Estimation by convergence. 
    
    The code is translated to Numpyro from Christian Breinholt's Pytorch implementation."""
    
    terms = 51
    lam = np.abs(lam) + 1e-12
    m = np.arange(0, terms)    
    log_binom = gammaln(2*m+1) - 2*gammaln(m+1) 
    
    logC = log_binom*np.ones((len(k1), terms)) + m*np.log((lam**2)/(4*k1*k2))[:,None] + log_im(m, k1) + log_im(m, k2)
    
    return - np.log(4*np.pi**2) - logsumexp(logC, axis = -1)

## Bivariate Von Mises Distribution class

In [5]:
from numpyro.distributions.distribution import Distribution
import time
@copy_docs_from(Distribution)
class BivariateVonMises(Distribution):
    """
    Bivariate von Mises (Sine Model) distribution on the torus
    
    Modality:
        If lam^2 / (k1*k2) > 1, the distribution is bimodal, otherwise unimodal.
            - This distribution is only defined for some 'slightly' bimodal cases (alpha < -7)
    
    :param mu, nu: an angle in radians.
        - mu & nu can be any real number but are interpreted as 2*pi
    :param k1, k2: concentration parameters
        - This distribution is only defined for k1, k2 > 0
    :param lam: correlation parameter
        - Can be any real number, but is not defined for very bimodal cases (see 'Modality')        
    :param w: reparameterization parameter
        - Has to be between -1 and 1
    """
    arg_constraints = {'mu':  constraints.real, 
                       'nu':  constraints.real,
                       'k1':  constraints.positive, 
                       'k2':  constraints.positive, 
                       'lam': constraints.real}
    support = constraints.real
    has_rsample = False
    
    def __repr__(self):
        param = self.lam
        printing = ", lam: "
        if self.lam is None:
            param = self.w
            printing = ", w: "
        return str(type(self).__name__) + "(mu: " + str(self.mu) + ", nu: " + str(self.nu) + ", k1: " + str(self.k1) + ", k2: " + str(self.k2) + printing + str(param) + ")"

    def __init__(self, mu, nu, k1, k2, lam = None, w = None, validate_args = None):
        
        if lam is None == w is None:
            raise ValueError("Either `lam` or `w` must be specified, but not both.")
        elif w is None:
            self.mu, self.nu, self.k1, self.k2, self.lam = promote_shapes(mu, nu, k1, k2, lam)
            alpha = 0.5*(self.k1 - (self.lam)**2/self.k2) 
            if not np.all(alpha > -7): 
                raise ValueError("Distribution is too bimodal or has too high concentration while being bimodal.")
        
        elif lam is None:
            self.mu, self.nu, self.k1, self.k2, self.w = promote_shapes(mu, nu, k1, k2, w)
            self.lam = np.sqrt(self.k1*self.k2) * self.w
            alpha = 0.5*(self.k1 - (self.lam)**2/self.k2) 
            if not np.all(alpha > -7): 
                raise ValueError("Distribution is too bimodal or has too high concentration while being bimodal.")
                
        batch_shape = lax.broadcast_shapes(np.shape(self.mu), 
                                           np.shape(self.nu),
                                           np.shape(self.k1),
                                           np.shape(self.k2),
                                           np.shape(self.lam))
        event_shape = (2,)
        
        if isinstance(self.k1, int) or isinstance(self.k1, float):
            self.logC = log_C(np.array([self.k1]), np.array([self.k2]), np.array([self.lam]))
        else:
            self.logC = log_C(self.k1, self.k2, self.lam)
            
        super(BivariateVonMises, self).__init__(batch_shape, event_shape, validate_args)
    
    def sample(self, key, sample_shape = (1)):
        """ Harshinder Singh, Vladimir Hnizdo, and Eugene Demchuk
            Probabilistic model for twodependent circular variables.
            Biometrika, 89(3):719–723, 2002.
            
        marg: marginal distribution (using _acg_bound())
        cond: conditional distribution using a modified univariate von Mises (as described in Singh et al. (2002))
        
        The sampling method follows Christian Breinholt's Pytorch implementation. 
        """
        if isinstance(sample_shape, int):
            shape = tuple([sample_shape]) + self.batch_shape + self.event_shape
        elif len(sample_shape) == 1:
            shape = sample_shape + self.batch_shape + self.event_shape
        else:
            shape = tuple([reduce(lambda x, y: x*y, sample_shape)])  + self.batch_shape + self.event_shape
            
        x = np.empty(shape)*np.nan
      
        if isinstance(self.k1, int) or isinstance(self.k1, float):
            marg = _acg_bound(x[:,None], np.array([self.k1])[:,None], np.array([self.k2])[:,None], np.array([self.lam])[:,None], key) #Sampling from marginal distribution
            marg = (marg + self.mu + np.pi) % (2 * np.pi) - np.pi
            marg = np.squeeze(marg)
        elif len(self.k1) == 1:
            marg = _acg_bound(x, self.k1[:,None], self.k2[:,None], self.lam[:,None], key) #Sampling from marginal distribution
            marg = (marg + self.mu + np.pi) % (2 * np.pi) - np.pi
            marg = np.squeeze(marg)
        else:
            marg = _acg_bound(x, self.k1, self.k2, self.lam, key)   #Sampling from marginal distribution
            marg = (marg + self.mu + np.pi) % (2 * np.pi) - np.pi   #Applying mean angle
          
        alpha = np.sqrt(self.k2**2 + self.lam**2 * np.sin(marg - self.mu)**2) #Sampling from conditional distribution
        beta  = np.arctan(self.lam / self.k2 * np.sin(marg - self.mu))
        cond  = VonMises(self.nu + beta, alpha).sample(key)
                
        if isinstance(sample_shape, tuple):
            if hasattr(self.k1, '__len__'):
                if len(self.k1) > 1:
                    marg = marg.T.reshape((len(self.k1),) + sample_shape)
                    cond = cond.T.reshape((len(self.k1),) + sample_shape)
                else: 
                    marg = marg.reshape(sample_shape)
                    cond = cond.reshape(sample_shape)   
            
            else: 
                marg = marg.reshape(sample_shape)
                cond = cond.reshape(sample_shape)

        return np.stack([marg, cond], -1)
    
    def expand(self, batch_shape):
        validate_args = self.__dict__.get('_validate_args')
        mu = np.ones(batch_shape)*self.mu
        nu = np.ones(batch_shape)*self.nu
        k1 = np.ones(batch_shape)*self.k1
        k2 = np.ones(batch_shape)*self.k2

        if self.lam is not None:
            lam = np.ones(batch_shape)*self.lam
            w = None
        else:
            w = np.ones(batch_shape)*self.w
            lam = None
        
        return BivariateVonMises(mu, nu, k1, k2, lam = lam, w = w, validate_args=validate_args)
    
    @validate_sample
    def log_prob(self, angles):
        """ Actual likelihood function, log joint distribution of phi and psi.
        The code was translated from Christian Breinholt's Pytorch implementation"""
            
        phi = angles[...,0]
        psi = angles[...,1]
        
        if hasattr(self.k1, '__len__'):
            if len(self.k1) > 1: 
                N = angles[...,0].shape[-1]
                k1 = self.k1.reshape((-1,) + (1,)*N)
                k2 = self.k2.reshape((-1,) + (1,)*N)
                mu = self.mu.reshape((-1,) + (1,)*N)
                nu = self.nu.reshape((-1,) + (1,)*N)
                lam = self.lam.reshape((-1,) + (1,)*N)
                logC = self.logC.reshape((-1,) + (1,)*N)
                
                log_prob = k1*np.cos(phi - mu) + k2*np.cos(psi - nu)
                log_prob += lam*np.sin(phi - mu)*np.sin(psi - nu)
                log_prob += logC
                return log_prob
            else:
                log_prob = self.k1*np.cos(phi - self.mu) + self.k2*np.cos(psi - self.nu)
                log_prob += self.lam*np.sin(phi - self.mu)*np.sin(psi - self.nu)
                log_prob += self.logC
                return log_prob
        else:
            phi = phi.flatten()
            psi = psi.flatten()
            log_prob = self.k1*np.cos(phi - self.mu) + self.k2*np.cos(psi - self.nu)
            log_prob += self.lam*np.sin(phi - self.mu)*np.sin(psi - self.nu)
            log_prob += self.logC
            return log_prob.reshape(angles[...,0].shape)


In [6]:
def Ramachandran_plot(data_angles, c):
    import matplotlib.pyplot as plt
    from matplotlib.colors import LogNorm
    degrees = np.rad2deg(data_angles) 
    phi = degrees[:,0] 
    psi = degrees[:,1] 
    plt.figure(figsize=(7, 6))
    plt.hist2d(phi, psi, bins = 200, norm = LogNorm(), cmap = plt.cm.jet )
    plt.suptitle("Ramachandran plot, {} samples from Bivariate von Mises".format(n))
    plt.title("mu = {}, nu = {}, k1 = {}, k2 = {}, lam = {}".format(mu, nu, k1, k2, lam))
    plt.xlabel('φ')
    plt.ylabel('ψ')    
    plt.xlim(-c, c)
    plt.ylim(-c, c)
    plt.colorbar();

## Four parallel distributions 
– Sampling with sample shape (10, 2) and calculating log probability for samples

In [7]:
mu_np = np.array([0., 0.2, 1.0, 0])
nu_np = np.array([0., 0.2, 1.0, 0])
k1_np = np.array([1., 70., 1, 70])
k2_np = np.array([1., 60., 1, 70])
lam_np = np.array([1., 15., 1, 6])

rng_key = random.PRNGKey(190)
bvms_b = BivariateVonMises(mu = mu_np, nu = nu_np, k1 = k1_np, k2 = k2_np, lam = lam_np)
angles_parallel = bvms_b.sample(rng_key, (10,2))
bvms_b.log_prob(angles_parallel)

DeviceArray([[[-3.7041    , -3.493853  ],
              [-3.6017377 , -2.2544556 ],
              [-3.9106045 , -5.5328746 ],
              [-2.5888896 , -3.006098  ],
              [-2.5731888 , -3.025478  ],
              [-2.344228  , -2.2679343 ],
              [-3.013022  , -4.227346  ],
              [-2.8640258 , -2.2765293 ],
              [-4.976006  , -2.490044  ],
              [-2.864221  , -2.9636009 ]],

             [[ 1.5474396 ,  2.1278381 ],
              [ 1.6512756 ,  2.2167816 ],
              [ 1.7980347 , -0.12492371],
              [-0.00796509,  1.9963226 ],
              [ 2.123352  ,  0.77153015],
              [ 1.9212341 ,  1.0536957 ],
              [-0.42440796,  0.67684937],
              [ 2.2994995 , -1.3942413 ],
              [ 1.6056824 ,  0.13626862],
              [ 1.1121063 ,  2.1580353 ]],

             [[-3.1141508 , -2.6420875 ],
              [-4.7155704 , -2.994446  ],
              [-5.4422064 , -2.2678635 ],
              [-2.9139745 , -3

## One distributions – initialising by inputting float/int values instead of np.arrays

In [8]:
mu = 0.
nu = 0.
k1 = 60.
k2 = 70.
lam = 15.
bvms_a = BivariateVonMises(mu = mu, nu = nu, k1 = k1, k2 = k2, lam = lam)
rng_key = random.PRNGKey(190)

n = 100000
start = time.time()
angles = bvms_a.sample(rng_key, (10,2))
end = time.time()

print(end-start)
bvms_a.log_prob(angles)

3.543272018432617


DeviceArray([[ 2.1934967 ,  1.5111694 ],
             [ 0.78448486,  2.1410522 ],
             [ 1.767456  ,  2.045288  ],
             [ 0.78352356,  0.79045105],
             [ 0.22862244,  1.3271484 ],
             [ 1.6169128 ,  1.203064  ],
             [ 1.8097839 ,  1.9964752 ],
             [ 2.0511017 ,  2.1817474 ],
             [ 1.5403137 ,  1.789917  ],
             [-2.4478607 ,  1.0534668 ]], dtype=float32)

## Initialising one distribution – the parameters are given as numpy arrays

In [9]:
mu = np.array([0])
nu = np.array([0])
k1 = np.array([60])
k2 = np.array([70])
lam = np.array([15])
bvms_a = BivariateVonMises(mu = mu, nu = nu, k1 = k1, k2 = k2, lam = lam)
rng_key = random.PRNGKey(190)

n = (100000, 2, 4)
start = time.time()
angles = bvms_a.sample(rng_key, n)
end = time.time()

print(end-start)

bvms_a.log_prob(angles)

6.044559001922607


DeviceArray([[[ 2.11438   ,  0.79042053,  1.8670349 ,  0.4592743 ],
              [ 1.5663757 , -3.3298264 ,  1.8549805 ,  1.0588074 ]],

             [[ 2.2818146 ,  1.4681396 ,  1.618042  ,  2.0597076 ],
              [ 0.9524536 , -0.49378204,  1.9419403 , -7.6818085 ]],

             [[ 2.0222168 ,  0.67692566,  1.8776093 ,  1.075531  ],
              [ 2.0820465 ,  2.296341  ,  0.99017334,  1.7017059 ]],

             ...,

             [[ 2.0026398 ,  1.7684784 ,  2.0164642 ,  2.1496887 ],
              [-0.7746048 ,  2.0533905 ,  1.1659546 ,  1.0926361 ]],

             [[ 2.0525513 ,  1.295105  ,  1.5915985 ,  1.1075897 ],
              [ 1.1691742 ,  1.4136047 ,  2.1525269 ,  2.246231  ]],

             [[ 2.0267487 ,  0.89372253,  1.9060516 , -0.22383118],
              [ 1.497757  ,  1.9042053 , -0.39237213,  0.8095703 ]]],            dtype=float32)